<img src='img/anaconda-logo.png' align='left' style="padding:10px">
<br>
*Copyright Continuum 2012-2016 All Rights Reserved.*

# Accelerate Feature Detection in Image Processing

## Table of Contents
* [CENSURE Keypoints Detector](#CENSURE-Keypoints-Detector)
	* [Detect keypoints](#Detect-keypoints)
	* [Extract & match descriptors at keypoints](#Extract-&-match-descriptors-at-keypoints)
	* [Estimate Transformation](#Estimate-Transformation)


## Set-up

In [ ]:
from __future__ import print_function, division
import skimage
import skimage.io

from skimage import data 
from skimage.transform import AffineTransform, warp

from skimage.color import rgb2gray

import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

# CENSURE Keypoints Detector

In [ ]:
# Replaces `from skimage.feature import CENSURE`
from accelerate_skimage.censure import CENSURE

## Detect keypoints

Original image

In [ ]:
img_orig = rgb2gray(data.astronaut())

Detect keypoints

In [ ]:
# detect
detector = CENSURE(mode="STAR")
detector.detect(img_orig)

In [ ]:
# copy the data
keypts_orig = detector.keypoints.copy()
scales_orig = detector.scales.copy()

In [ ]:
# plot the data
plt.imshow(img_orig, cmap=plt.cm.gray)
plt.axis('off')
plt.scatter(detector.keypoints[:, 1], detector.keypoints[:, 0],
            2 ** detector.scales, facecolors='none', edgecolors='r');

Warped image

In [ ]:
tform = AffineTransform(scale=(1.05, 1.05), rotation=0.15, translation=(20, -70))
img_warp = warp(img_orig, tform)

In [ ]:
# detect
detector.detect(img_warp)

# copy the data
keypts_warp = detector.keypoints.copy()
scales_warp = detector.scales.copy()

# plot
plt.imshow(img_warp, cmap=plt.cm.gray)
plt.axis('off')
plt.scatter(detector.keypoints[:, 1], detector.keypoints[:, 0],
            2 ** detector.scales, facecolors='none', edgecolors='r');

## Extract & match descriptors at keypoints 

In [ ]:
from skimage.feature import BRIEF, match_descriptors, plot_matches

Extract descriptors at the keypoints from the original image

In [ ]:
extractor = BRIEF(patch_size=10)
extractor.extract(img_orig, keypts_orig)
desc_orig = extractor.descriptors
len(desc_orig)

Extract descriptors at the keypoints from the wrapped image

In [ ]:
extractor.extract(img_warp, keypts_warp)
desc_warp = extractor.descriptors
len(desc_warp)

Find matching descriptors from the two images

In [ ]:
matches = match_descriptors(desc_orig, desc_warp)
len(matches)

In [ ]:
plot_matches(plt, img_orig, img_warp, keypts_orig, keypts_warp, matches)

## Estimate Transformation 

We will use RANSAC to estimate the affine transformation used to produce the wrapped image.

In [ ]:
from skimage.measure import ransac

Prepare the keypoints as a numpy array

In [ ]:
src = []
dst = []

for s, d in matches:
    src.append(keypts_orig[s])
    dst.append(keypts_warp[d])
    
src = np.asarray(src)
dst = np.asarray(dst)

Run RANSAC to estimate the transformation

In [ ]:
%%time
model, inliers = ransac((src, dst), AffineTransform, min_samples=5,
                        residual_threshold=1, max_trials=30000)

In [ ]:
print('inliers', np.nonzero(inliers == True)[0].size)
print(model.scale, model.translation, model.rotation)

Create a new warp image using the esitmated transformation

In [ ]:
img_result = warp(img_orig, model)

In [ ]:
plt.imshow(img_result, cmap=plt.cm.gray)
plt.axis('off')

Plot the difference

In [ ]:
plt.imshow(img_result - img_warp, cmap=plt.cm.gray)
plt.axis('off')

Show descriptors that matches between the original and wrapped image

In [ ]:
inlier_idxs = np.nonzero(inliers)[0]
plot_matches(plt, img_orig, img_warp, src, dst,
             np.column_stack((inlier_idxs, inlier_idxs)), matches_color='b')

Show descriptors that didn't match between the original and wrapped image

In [ ]:
outlier_idxs = np.nonzero(inliers == False)[0]
plot_matches(plt, img_orig, img_warp, src, dst,
             np.column_stack((outlier_idxs, outlier_idxs)), matches_color='r')

---
*Copyright Continuum 2012-2016 All Rights Reserved.*